# CMS OPEN PAYMENT DATA 2016


In [1]:
import pandas as pd
import numpy as np

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Process CVS In Chunks

In [3]:
# IMPORT CSV
chunksize = 500000
iter_csv = pd.read_csv(
    "DataSets/CMS_openpaymens/2016payments.csv",
    header=0,
    usecols=[
        "Physician_Profile_ID",
        "Physician_First_Name",
        "Physician_Middle_Name",
        "Physician_Last_Name",
        "Physician_Name_Suffix",
        "Recipient_City",
        "Recipient_State",
        "Physician_Specialty",
        "Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name",
        "Total_Amount_of_Payment_USDollars",
        "Number_of_Payments_Included_in_Total_Amount",
        "Nature_of_Payment_or_Transfer_of_Value",
        "Physician_Ownership_Indicator",
        "Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_1",
        "Product_Category_or_Therapeutic_Area_1",
    ],
    iterator=True,
    low_memory=False,
    chunksize=chunksize,
)

# PROCESS CSV
counter = 0
for chunk in iter_csv:

    # Condense physician column names and remove separate name columns
    chunk["physician_name"] = (
        chunk[chunk.columns[1:4]]
        .apply(lambda x: " ".join(x.dropna().astype(str)), axis=1)
        .str.title()
    )
    chunk = chunk.drop(
        [
            "Physician_First_Name",
            "Physician_Middle_Name",
            "Physician_Last_Name",
            "Physician_Name_Suffix",
        ],
        axis=1,
    )  # drop separate name columns

    # Rename columns
    chunk.columns = [
        "physician_id",
        "physician_city",
        "physician_state",
        "physician_specialty",
        "company_name",
        "total_payment",
        "num_payment_per_total",
        "payment_type",
        "physician_ownership",
        "product_type",
        "product_specialty",
        "physician_name",
    ]

    # Drop rows with missing data
    chunk.replace("", np.nan, inplace=True)
    must_have_data = [
        "physician_id",
        "physician_state",
        "physician_specialty",
        "company_name",
        "total_payment",
        "num_payment_per_total",
        "payment_type",
        "physician_ownership",
        "product_type",
    ]

    chunk = chunk.dropna(subset=must_have_data, how="any")

    # REFORMAT DATA OF INTEREST
    # Isolate only physicians (exclude dentists, etc.)
    chunk["physician_specialty"] = chunk["physician_specialty"].apply(
        str
    )  # convert to str
    chunk = chunk[
        chunk["physician_specialty"].str.contains("Allopathic")
    ]  # isolate doctors
    chunk = chunk[
        chunk["product_type"].str.contains("Device|Drug|Biological")
    ]  # isolate product type

    # Change City to title
    chunk["physician_city"] = chunk["physician_city"].apply(str.title)

    # Change physician id from float to string
    chunk["physician_id"] = chunk["physician_id"].apply(str)

    # Parse Physician_Specialty column to identify general specialty
    chunk["physician_specialty"] = chunk["physician_specialty"].apply(
        lambda x: str(x).split("|")[1]
    )

    # Reformat payment_type column
    chunk.payment_type = chunk.payment_type.str.replace(
        r"(^.*Compensation.*$)", "Guest Lecture"
    )
    chunk.payment_type = chunk.payment_type.str.replace(
        r"(^.*Honoraria.*$)", "Guest Lecture"
    )
    chunk.payment_type = chunk.payment_type.str.replace(r"(^.*Grant.*$)", "Education")
    chunk.payment_type = chunk.payment_type.str.replace(
        r"(^.*Charitable.*$)", "Education"
    )
    chunk.payment_type = chunk.payment_type.str.replace(r"(^.*Gift.*$)", "Education")
    chunk.payment_type = chunk.payment_type.str.replace(
        r"(^.*Entertainment.*$)", "Food and Beverage"
    )
    chunk.payment_type = chunk.payment_type.str.replace(
        r"(^.*ownership.*$)", "Consulting Fee"
    )

    # Store Data
    if counter == 0:
        df = chunk
    else:
        df = pd.concat([df, chunk])
    counter += 1

    # Size of df
    print(df.shape)

(401700, 12)
(865652, 12)
(1295964, 12)
(1642424, 12)
(2055460, 12)
(2482746, 12)
(2951883, 12)
(3410892, 12)
(3814367, 12)
(4228035, 12)
(4638937, 12)
(5056001, 12)
(5499630, 12)
(5925150, 12)
(6353865, 12)
(6762890, 12)
(7119982, 12)
(7552597, 12)
(7981197, 12)
(8380384, 12)
(8848819, 12)
(9335777, 12)
(9831669, 12)
(9895605, 12)


<IPython.core.display.Javascript object>

## Summary Spreadsheets For R

In [4]:
new_df = df


<IPython.core.display.Javascript object>

In [5]:
# ENTIRE DATASET: Descriptive Summary By Specialty
specialty_totalpayments = new_df.groupby("physician_specialty")["total_payment"].sum()
specialty_npayments = new_df.groupby("physician_specialty")[
    "num_payment_per_total"
].sum()
specialty_ndoctors = new_df.groupby("physician_specialty")["physician_id"].nunique()


# Combine Descriptive Values
result1 = pd.merge(
    specialty_totalpayments, specialty_npayments, on="physician_specialty"
)
result2 = pd.merge(result1, specialty_ndoctors, on="physician_specialty")
results2 = result2.sort_values("total_payment", ascending=False)

# Export Data
results2.to_csv(
    "DataSets/openpayment_specialty_descriptive.csv",
    header=["total_payments", "npayments", "ndoctors"],
)

<IPython.core.display.Javascript object>

In [6]:
# ENTIRE DATASET: Descriptive Summary By Specialty
product_totalpayments = new_df.groupby("product_type")["total_payment"].sum()
product_npayments = new_df.groupby("product_type")["num_payment_per_total"].sum()
product_ndoctors = new_df.groupby("product_type")["physician_id"].nunique()


# Combine Descriptive Values
result1 = pd.merge(product_totalpayments, product_npayments, on="product_type")
result2 = pd.merge(result1, product_ndoctors, on="product_type")
results2 = result2.sort_values("total_payment", ascending=False)

# Export Data
results2.to_csv(
    "DataSets/openpayment_products_descriptive.csv",
    header=["total_payments", "npayments", "ndoctors"],
)

<IPython.core.display.Javascript object>